Lucy de Haan, Sean Sun, Faye Shipp, Ryan Wang

Title : An Exploration of PLAICRAFT Data
-----------------------------------------

**Introduction**

 - provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report
 - clearly state the question you tried to answer with your project
 - identify and fully describe the dataset that was used to answer the question

**Methods and Results**

 - describe the methods you used to perform your analysis from beginning to end that narrates the analysis code.
 - your report should include code which:
    - loads data 
    - wrangles and cleans the data to the format necessary for the planned analysis
    - performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
    - creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
    - performs the data analysis
    - creates a visualization of the analysis 
* note: all figures should have a figure number and a legend

**Discussion**

 - summarize what you found
 - discuss whether this is what you expected to find?
 - discuss what impact could such findings have?
 - discuss what future questions could this lead to?

**References**

 - You may include references if necessary, as long as they all have a consistent citation style.

(I don't think we need this but I'll leave it here in case)